In [ ]:
#
# Representation using splines for edges
#
import polars   as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()

# Render Operation
def svgSplineRender(_df_):
    _ops_ = [pl.lit('<path d="M '), pl.col('x0'), pl.lit(' '), pl.col('y0'), pl.lit(' C '),
             pl.col('xa'), pl.lit(' '), pl.col('ya'), pl.lit(' '),
             pl.col('xb'), pl.lit(' '), pl.col('yb'), pl.lit(' '),
             pl.col('x1'), pl.lit(' '), pl.col('y1'), pl.lit('" fill="none" stroke="#000000" stroke-width="0.002" />')]
    x0, y0, x1, y1 = min(_df_['x0'].min(), _df_['x1'].min()), min(_df_['y0'].min(), _df_['y1'].min()), max(_df_['x0'].max(), _df_['x1'].max()), max(_df_['y0'].max(), _df_['y1'].max())
    svg = [f'<svg x="0" y="0" width="768" height="768" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">']
    svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="#ffffff" />')
    _df_ = _df_.with_columns(pl.concat_str(_ops_).alias('svg'))
    svg.extend(_df_['svg'].to_list())
    svg.append('</svg>')
    return ''.join(svg)

edges_filename  = '../../data/stanford/facebook/414.edges'
layout_filename = '../../data/stanford/facebook/414.rt_layout.parquet'

_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df  = pl.DataFrame(_lu_)

g   = rt.createNetworkXGraph(df, [('fm','to')])
pos = nx.spring_layout(g)

_nodes_                   = list(pos.keys())
_xs_, _ys_, _ws_, _ns_ = [], [], [], []
for i in range(len(_nodes_)):
    _xs_   .append(pos[_nodes_[i]][0])
    _ys_   .append(pos[_nodes_[i]][1])
    _ws_   .append(g.degree(_nodes_[i]))
    _ns_   .append(_nodes_[i])
_df_         = pl.DataFrame({'x':_xs_, 'y':_ys_, 'w':_ws_, 'node':_ns_})
_df_results_ = rt.uniformSampleDistributionInScatterplotsViaSectorBasedTransformation(_df_, 'x', 'y', wgt_field='w', iterations=128)
_pos_ = {}
for i in range(len(_df_results_)): _pos_[_df_results_['node'][i]] = (_df_results_['x'][i], _df_results_['y'][i])
rt.tile([rt.link(df, [('fm','to')], pos,   w=600, h=600, link_shape='curve', link_size=1),
         rt.link(df, [('fm','to')], _pos_, w=600, h=600, link_shape='curve', link_size=1)], spacer=10)